<a href="https://colab.research.google.com/github/ayush12gupta/CNN_models/blob/master/MobileNet_v2/MobileNetv2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import torch.nn as nn
import math
import torch
from collections import OrderedDict
from torch.nn import init
import torchvision
import torchvision.transforms as transforms
import torch.optim as optim
import numpy as np
import torch.nn.functional as F


In [0]:
def _make_divisible(v, divisor, min_value=None):
    if min_value is None:
        min_value = divisor
    new_v = max(min_value, int(v + divisor / 2) // divisor * divisor)
    # Make sure that round down does not go down by more than 10%.
    if new_v < 0.9 * v:
        new_v += divisor
    return new_v

In [0]:
def cov_block(in_fil, filters, kernel, strides):
    return nn.Sequential(
        nn.Conv2d(in_fil,filters,kernel,strides,bias=False),
        nn.BatchNorm2d(filters),
        nn.ReLU6(inplace=True)
    )

In [0]:
''' 
  alpha: integer, width multiplier
  p: resolution multiplier
'''  
class bottleneck(nn.Module):
  def __init__(self,in_fil, filters, p, alpha, stride):
    super(bottleneck,self).__init__()
    self.stride = stride
    assert stride in [1,2]
    self.in_fil = in_fil
    self.filters = filters
    self.depth = in_fil*p
    self.filters = int(alpha*filters)
    self.cov1 = cov_block(self.in_fil,self.depth,1,1)
    self.cov2 = nn.Conv2d(self.depth,self.depth,kernel_size=3, stride=self.stride, padding=1, bias=False, groups=self.depth)
    self.bn1 = nn.BatchNorm2d(self.depth)
    self.cov3 = nn.Conv2d(self.depth, self.filters, kernel_size=1, bias=False)
    self.bn2 = nn.BatchNorm2d(self.filters)
    self.activation = nn.ReLU6(inplace=True)

  def forward(self,x):
    residual = x
    # Pointwise
    out = self.cov1(x)
    # Depthwise
    out = self.cov2(out)  
    out = self.bn1(out)
    out = self.activation(out)
    # Pointwise
    out = self.cov3(out)
    out = self.bn2(out)
    # Make residual
    if self.stride == 1 and self.in_fil == self.filters:
      out += residual
    
    return out  

In [0]:
class MobileNetv2(nn.Module):
  def __init__(self,alpha=1.0, input_size=224, p=6, in_channels=3, num_classes=1000):
    super(MobileNetv2,self).__init__()
    
    self.alpha = alpha    
    self.p = p
    self.activation = nn.ReLU6(inplace=True)
    self.num_classes=num_classes
    self.channels = [32, 16, 24, 32, 64, 96, 160, 320]
    self.c = [_make_divisible(ch * self.alpha, 8) for ch in self.channels]
    self.n = [ 1, 1, 2, 3, 4, 3, 3, 1]
    self.s = [2, 1, 2, 2, 2, 1, 2, 1]
    self.cov1 = nn.Conv2d(in_channels, self.c[0], kernel_size=3, bias=False, stride=self.s[0], padding=1)
    self.bn1 = nn.BatchNorm2d(self.c[0])
   # self.inverted_residual = self.inverted_residual()
    self.bottlenecks = self._make_bottlenecks()
    self.last_conv_out_ch = 1280 if self.alpha <= 1 else _make_divisible(1280 * self.alpha, 8)
    self.conv_last = nn.Conv2d(self.c[-1], self.last_conv_out_ch, kernel_size=1, bias=False)
    self.bn_last = nn.BatchNorm2d(self.last_conv_out_ch)
    self.avgpool = nn.AdaptiveAvgPool2d(1)
    self.dropout = nn.Dropout(p=0.2, inplace=True)  # confirmed by paper authors
    self.fc = nn.Linear(self.last_conv_out_ch, self.num_classes)
    self.para_init()

  def _make_stage(self, inplanes, outplanes, n, stride, t, stage):
        modules = OrderedDict()
        stage_name = "LinearBottleneck{}".format(stage)

        # First module is the only one utilizing stride
        first_module = bottleneck(filters= outplanes,in_fil=inplanes, stride=stride, p=t,alpha=self.alpha)
        modules[stage_name + "_0"] = first_module

        # add more LinearBottleneck depending on number of repeats
        for i in range(n - 1):
            name = stage_name + "_{}".format(i + 1)
            module = bottleneck(in_fil= outplanes,filters=outplanes, stride=1, p=t,alpha=self.alpha)
            modules[name] = module

        return nn.Sequential(modules)

  def _make_bottlenecks(self):
        modules = OrderedDict()
        stage_name = "Bottlenecks"

        # First module is the only one with t=1
        bottleneck1 = self._make_stage(inplanes=self.c[0],outplanes =self.c[1], n=self.n[1], stride=self.s[1], t=1,
                                       stage=0)
        modules[stage_name + "_0"] = bottleneck1

        # add more LinearBottleneck depending on number of repeats
        for i in range(1, len(self.c) - 1):
            name = stage_name + "_{}".format(i)
            module = self._make_stage(inplanes=self.c[i], outplanes=self.c[i + 1], n=self.n[i + 1],
                                      stride=self.s[i + 1],
                                      t=self.p, stage=i)
            modules[name] = module

        return nn.Sequential(modules)

  def para_init(self):
        for m in self.modules():
            if isinstance(m, nn.Conv2d):
                init.kaiming_normal_(m.weight, mode='fan_out')
                if m.bias is not None:
                    init.constant_(m.bias, 0)
            elif isinstance(m, nn.BatchNorm2d):
                init.constant_(m.weight, 1)
                init.constant_(m.bias, 0)
            elif isinstance(m, nn.Linear):
                init.normal_(m.weight, std=0.001)
                if m.bias is not None:
                    init.constant_(m.bias, 0)        
  def forward(self, x):
        x = self.cov1(x)
        x = self.bn1(x)
        x = self.activation(x)

        x = self.bottlenecks(x)
        x = self.conv_last(x)
        x = self.bn_last(x)
        x = self.activation(x)

        # average pooling layer
        x = self.avgpool(x)
        x = self.dropout(x)

        # flatten for input to fully-connected layer
        x = x.view(x.size(0), -1)
        x = self.fc(x)
        return F.log_softmax(x, dim=1)                    

In [6]:
model1 = MobileNetv2()
print(model1)

MobileNetv2(
  (activation): ReLU6(inplace=True)
  (cov1): Conv2d(3, 32, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
  (bn1): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (bottlenecks): Sequential(
    (Bottlenecks_0): Sequential(
      (LinearBottleneck0_0): bottleneck(
        (cov1): Sequential(
          (0): Conv2d(32, 32, kernel_size=(1, 1), stride=(1, 1), bias=False)
          (1): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
          (2): ReLU6(inplace=True)
        )
        (cov2): Conv2d(32, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), groups=32, bias=False)
        (bn1): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (cov3): Conv2d(32, 16, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (bn2): BatchNorm2d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (activation): ReLU6(inplace=True)
      )
    

In [7]:
batch_size = 128

transform = transforms.Compose([transforms.Resize(256),transforms.CenterCrop(224),transforms.ToTensor(), transforms.Normalize((.5, .5, .5), (.5, .5, .5)) ])

trainset = torchvision.datasets.CIFAR10(root = './data', download = True, train = True, transform = transform)
trainloader = torch.utils.data.DataLoader(trainset, batch_size = batch_size, shuffle = True, num_workers=2)

testset = torchvision.datasets.CIFAR10(root = './data', download = True, train = False, transform = transform)
testloader = torch.utils.data.DataLoader(testset, batch_size = batch_size, shuffle = True, num_workers=2) # num workers is used to pre process data

Extracting ./data/cifar-10-python.tar.gz to ./data
Files already downloaded and verified


In [0]:
model = MobileNetv2(num_classes=10).cuda()
learning_rate = 0.01
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr = learning_rate)
scheduler = optim.lr_scheduler.MultiStepLR(optimizer, milestones=[30,50,80], gamma=0.3)
device = 'cuda'

In [9]:
model.train()
print('Training Started')
for epoch in range(35):

    running_loss = 0.0
    correct, total = 0.0, 0.0
    correct_epoch, total_epoch = 0.0, 0.0
    scheduler.step()
    for i,data in enumerate(trainloader):

        inp,lab = data[0].to(device),data[1].to(device)

        optimizer.zero_grad()

        out = model(inp)
        loss = criterion(out,lab)
        
        loss.backward()
        optimizer.step()
        
        running_loss += loss.item()
        c = (torch.argmax(out,1)==lab)
        correct+=torch.sum(c)
        total += (batch_size)
        correct_epoch += torch.sum(c)
        total_epoch += (batch_size)
        
        if i % 200 == 199:    # print every 200 mini-batches
            print('[%d, %5d] loss: %.7f accuracy: [%d, %d] (%d %%)' % (epoch + 1, i + 1, running_loss / 200, correct, total, float(100*(correct/total))))
            running_loss = 0.0
            correct = 0.0
            total = 0.0
            #print('Epoch: {}   loss: {}  accuracy: {}'.format(epoch,loss,(float(correct/total))))

Training Started


/usr/local/lib/python3.6/dist-packages/torch/optim/lr_scheduler.py:122: UserWarning: Detected call of `lr_scheduler.step()` before `optimizer.step()`. In PyTorch 1.1.0 and later, you should call them in the opposite order: `optimizer.step()` before `lr_scheduler.step()`.  Failure to do this will result in PyTorch skipping the first value of the learning rate schedule. See more details at https://pytorch.org/docs/stable/optim.html#how-to-adjust-learning-rate
  "https://pytorch.org/docs/stable/optim.html#how-to-adjust-learning-rate", UserWarning)


[1,   200] loss: 2.0499135 accuracy: [5927, 25600] (23 %)
[2,   200] loss: 1.4832839 accuracy: [11423, 25600] (44 %)
[3,   200] loss: 1.1438632 accuracy: [14983, 25600] (58 %)
[4,   200] loss: 0.9183256 accuracy: [17154, 25600] (67 %)
[5,   200] loss: 0.7481966 accuracy: [18936, 25600] (73 %)
[6,   200] loss: 0.6581369 accuracy: [19738, 25600] (77 %)
[7,   200] loss: 0.5814725 accuracy: [20411, 25600] (79 %)
[8,   200] loss: 0.5274357 accuracy: [20856, 25600] (81 %)
[9,   200] loss: 0.4934566 accuracy: [21247, 25600] (82 %)
[10,   200] loss: 0.4456606 accuracy: [21647, 25600] (84 %)
[11,   200] loss: 0.4126504 accuracy: [21964, 25600] (85 %)
[12,   200] loss: 0.3824828 accuracy: [22168, 25600] (86 %)
[13,   200] loss: 0.3578090 accuracy: [22374, 25600] (87 %)
[14,   200] loss: 0.3299786 accuracy: [22665, 25600] (88 %)
[15,   200] loss: 0.3179591 accuracy: [22759, 25600] (88 %)
[16,   200] loss: 0.2944810 accuracy: [22999, 25600] (89 %)
[17,   200] loss: 0.2623873 accuracy: [23285, 2560

In [0]:
torch.save(model.state_dict(), '/content/mobilenetv2.pt')


In [11]:
print(correct_epoch/total_epoch)

tensor(0.9925, device='cuda:0')


In [13]:
device='cpu'
model.to(device)
model.eval()
running_loss = 0.0
correct, total = 0.0, 0.0
#correct_epoch, total_epoch = 0, 0
for i, data in enumerate(testloader):
    inp,lab = data[0].to(device),data[1].to(device)
    out = model(inp) 
    c = (torch.argmax(out,1)==lab)
    correct+=torch.sum(c)
    total += batch_size
    
print('accuracy: ',100*correct/total,'',correct)

accuracy:  tensor(86.0265)  tensor(8699.)
